# Working with OpenAI GPT models

just like any other APIs, you can send a request to openAI chatGPT server and get the response back from your query.

[<img src="https://res.cloudinary.com/practicaldev/image/fetch/s--6UwyTHKO--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_800/https://dev-to-uploads.s3.amazonaws.com/uploads/articles/t637s1yazyyxfl31ymmq.jpg">](https://res.cloudinary.com/practicaldev/image/fetch/s--6UwyTHKO--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_800/https://dev-to-uploads.s3.amazonaws.com/uploads/articles/t637s1yazyyxfl31ymmq.jpg)


You need to:
* [Set up an account](https://auth0.openai.com/u/signup/identifier?state=hKFo2SBLZVEyMlJSRDNkbWVMUWVYdU5SVGZKQWltY016ek1POaFur3VuaXZlcnNhbC1sb2dpbqN0aWTZIEJxeTRsb191RnZySEV0b2dlYnRZdGNzQWpZdkRWZjI4o2NpZNkgRFJpdnNubTJNdTQyVDNLT3BxZHR3QjNOWXZpSFl6d0Q)
* [Get an API Key](https://platform.openai.com/api-keys)
* Add money!

In [ ]:
!pip install --upgrade openai

In [ ]:
from openai import OpenAI
from datasets import load_dataset

import random
import pandas as pd
from pprint import pprint

# I created a local config.py file to manage my secret keys
from config import API_KEY 

In [ ]:
client = OpenAI(api_key=API_KEY)

### Text classification with a LLM

In [ ]:
# download and cache the dataset:
raw_datasets = load_dataset("imdb")

In [ ]:
raw_datasets['train']['text'][1]

In [ ]:
raw_datasets['train']['label'][1]

# Add info on temp and other params

In [ ]:
params ={
    "model_name": "gpt-3.5-turbo",
    "temperature": 0.1,
    "max_tokens":256
}

def classifier(input_text, parameters, client=client):
    
    
    messages=[
    {"role": "system", "content": "You are a useful assitant for the imdb website. You should read the submitted movie review by a user below and decide if it is a positive or negative. return the result with 0 or 1 for negative and positive respectively"},
    {"role": "user", "content": input_text}
    ]
    
    response = client.chat.completions.create(
        model=parameters["model_name"],
        messages=messages,
        temperature=parameters["temperature"], 
        max_tokens=parameters["max_tokens"],
    )

    return response.choices[0].message.content

In [ ]:
classifier(raw_datasets['train']['text'][1], params)

I can now run the same query on all the rows and get the response

In [ ]:
# select random 20 review and their label
random_idx = random.sample(range(1, 25000), 20)


sel_text = [raw_datasets['train']['text'][i] for i in random_idx]
sel_labels = [raw_datasets['train']['label'][i] for i in random_idx]

In [ ]:
# turn to a dataframe to make it easier to see and manipulate data/
df = pd.DataFrame([sel_text, sel_labels]).T
df.columns = ['text', 'label']

df.head()

**pro tip**: **Partial Functions**

A partial function allows us to call a second function with fixed values in certain arguments.

In [ ]:
from functools import partial

classifier_pd = partial(classifier, parameters=params)


In [ ]:
%%time
df['predicted'] = df['text'].apply(classifier_pd)

You can't get away from data cleaning!!

In [ ]:
df['predicted'] = df['predicted'].apply(lambda x: '1' if "positive" in x else x)
df['predicted'] = df['predicted'].apply(lambda x: '0' if "negative" in x else x)

df[["label", "predicted"]] = df[["label", "predicted"]].apply(pd.to_numeric)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(df["label"], df["predicted"]))

# Langchain

LangChain provides many modules that can be used to build language model applications

In [ ]:
!pip install langchain
!pip install typing-inspect==0.8.0
!pip install typing_extensions==4.5.0

In [ ]:
from langchain.chat_models import ChatOpenAI

from config import API_KEY 

In [ ]:
# LLM Model: The language model is the core reasoning engine.
llm = ChatOpenAI(openai_api_key=API_KEY)


There are two types of language models:

* `LLM`: underlying model takes a string as input and returns a string
* `ChatModel`: underlying model takes a list of messages as input and returns a message

In [ ]:
from langchain.schema import HumanMessage

In [ ]:
review = raw_datasets['train']['text'][1]

text = f"""
You are a useful assitant for the imdb website.
You should read the submitted movie review by a user below and decide if it is a positive or negative.
Return the result with 0 or 1 for negative and positive respectively.

{review}
"""

messages = [HumanMessage(content=text)]
messages


The simplest way to call an LLM or ChatModel is using `.invoke()`

In [ ]:
llm.invoke(text)


## prompt template

When you don't want to pass user input directly into an LLM, you can add the it to a larger piece of text, called a `prompt template`. It provides additional context on the specific task at hand.

In the previous example, the text we passed to the model contained instructions. By using prompt templates, we only have to provide the review itself, without having to worry about giving the model instructions.

In [ ]:
from langchain.prompts import PromptTemplate


In [ ]:
prompt = PromptTemplate.from_template("""
You are a useful assitant for the imdb website.
You should read the submitted movie review by a user below and decide if it is a positive or negative.
Return the result with 0 or 1 for negative and positive respectively.

{review}
"""
)

prompt.format(review=raw_datasets['train']['text'][1])

## Output parser

`OutputParsers` convert the raw output of a language model into a format that can be used downstream.


In [ ]:
from langchain.schema import BaseOutputParser

class BooleanParser(BaseOutputParser):
    """Parse the output of an LLM call to 0 and 1."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        r = 0 if 'negative' in s else 1
        return r


## Putting it all together

We can now combine all these into one chain. This chain will take input variables, pass those to a prompt template to create a prompt, pass the prompt to a language model, and then pass the output through an (optional) output parser

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser


In [ ]:
class BooleanParser(BaseOutputParser):
    """Parse the output of an LLM call to 0 and 1."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        r = 0 if 'negative' in text else 1
        return r

prompt = PromptTemplate.from_template("""
You are a useful assitant for the imdb website.
You should read the submitted movie review by a user below and decide if it is a positive or negative.
Return the result with 0 or 1 for negative and positive respectively.

{review}
"""
)

llm = ChatOpenAI(openai_api_key=API_KEY)

In [ ]:
chain = prompt | llm | BooleanParser()

In [ ]:
chain.invoke({"review": raw_datasets['train']['text'][1]})

# [Code Assitant](https://python.langchain.com/docs/expression_language/cookbook/code_writing)

In [ ]:
!pip install langchain_experimental

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema.output_parser import StrOutputParser
from langchain_experimental.utilities import PythonREPL

from config import API_KEY 

In [7]:
template = """Write some python code to solve the user's problem. 

Return only python code in Markdown format, e.g.:

```python
....
```"""
prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

model = ChatOpenAI(api_key=API_KEY)

In [8]:
def _sanitize_output(text: str):
    _, after = text.split("```python")
    return after.split("```")[0]

In [9]:
chain = prompt | model | StrOutputParser() | _sanitize_output | PythonREPL().run

In [11]:
chain.invoke({"input": "use sklearn pipelines to create a xgboost classifier pipeline with scaling "})

'ModuleNotFoundError("No module named \'xgboost\'")'